In [40]:

from net import GaussianDiffusion
from net import EpsilonTheta
import torch
from torch.utils.data import DataLoader
import wandb
import math
import optuna
import os


from tqdm import tqdm
import os


In [41]:
def custom_collate_fn(batch):
    """
    Custom collate function to reshape data into [batch size, channels, size].
    """
    # Assuming your signals are originally in the shape [size]
    # and you want to add a single channel dimension
    signals = torch.stack([item['signals'] for item in batch]).unsqueeze(1)  # Adds a channel dimension
    gt = torch.stack([item['gt'] for item in batch])
    sc = torch.stack([item['sc'] for item in batch])
    
    return {'signals': signals, 'gt': gt, 'sc': sc}

In [42]:
def train_model(epochs, train_loader, num_batches_per_epoch, model, optimizer, 
                validation_iter=None, device='cuda', model_save_path='./model_checkpoints/', 
                log_interval=100):
    """
    A function to train the model and return the average validation loss.
    :param epochs: Number of epochs to train.
    :param train_loader: DataLoader for training data.
    :param num_batches_per_epoch: Number of batches in each epoch.
    :param model: The neural network model to train.
    :param optimizer: Optimizer used for training.
    :param validation_iter: DataLoader for validation data, if any.
    :param device: Device to train on, 'cuda' or 'cpu'.
    :param model_save_path: Path to save model checkpoints.
    :param log_interval: Interval to log training progress.
    
    :return: Average validation loss over the validation dataset.
    """
    losses_t = []
    if not os.path.exists(model_save_path):
        os.makedirs(model_save_path)

    for epoch in range(epochs):
        model.train()
        cumm_epoch_loss = 0.0

        with tqdm(train_loader, total=num_batches_per_epoch - 1) as it:
            for batch_no, data_entry in enumerate(it, start=1):
                optimizer.zero_grad()
                signals = data_entry['signals'].to(device)
                losses = model.log_prob(signals)
                cumm_epoch_loss += losses.item()

                avg_epoch_loss = cumm_epoch_loss / batch_no
                it.set_postfix({"epoch": f"{epoch + 1}/{epochs}", "avg_loss": avg_epoch_loss}, refresh=False)

                wandb.log({"train_loss": losses.item()})
                losses.backward()
                optimizer.step()
                #lr_scheduler.step()

                if num_batches_per_epoch == batch_no:
                    break

        losses_t.append(avg_epoch_loss)
        if (epoch + 1) % log_interval == 0:
            model_checkpoint_path = os.path.join(model_save_path, f'model_epoch_{epoch+1}.pth')
            torch.save(model.state_dict(), model_checkpoint_path)
            print(f'Model saved to {model_checkpoint_path}')

    # Validation loop
    if validation_iter is not None:
        model.eval()
        cumm_epoch_loss_val = 0.0
        with tqdm(validation_iter, total=num_batches_per_epoch - 1, colour="green") as it:
            for batch_no, data_entry in enumerate(it, start=1):
                signals = data_entry['signals']
                with torch.no_grad():
                    losses = model.log_prob(signals)

                cumm_epoch_loss_val += losses.item()
                avg_epoch_loss_val = cumm_epoch_loss_val / batch_no

                it.set_postfix({"epoch": f"{epoch + 1}/{epochs}", "avg_val_loss": avg_epoch_loss_val}, refresh=False)

        return avg_epoch_loss_val
    else:
        return sum(losses_t) / len(losses_t)


In [43]:
def objective(trial):
    # Hyperparameters to optimize
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    num_layers = trial.suggest_int('num_layers', 4, 16)
    residual_channels = trial.suggest_categorical('residual_channels', [16, 32, 64])
    dilation_cycle_length = trial.suggest_categorical('dilation_cycle_length', [1, 2, 4])

    nb_samples = 10000
    num_batches_per_epoch = math.ceil(nb_samples / batch_size)

    
    
    file_path = 'datasets/train_set.pth'
    dataset = torch.load(file_path)

    # DataLoader for training
    train_loader = DataLoader(dataset, batch_size=32, shuffle=True,collate_fn=custom_collate_fn)
    # Initialize model, optimizer, etc. using the suggested hyperparameters
    # For example:
    denoise_fn = EpsilonTheta(target_dim=[256],
                              residual_layers=num_layers,
                              residual_channels=residual_channels,
                              dilation_cycle_length=dilation_cycle_length)
    model = GaussianDiffusion(denoise_fn=denoise_fn, input_size=[256], beta_end=0.1, diff_steps=100, loss_type="l2", betas=None, beta_schedule="linear")
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    

    # Your training function should return the validation loss
    val_loss = train_model(100, train_loader, num_batches_per_epoch, model, optimizer, 
                validation_iter=None, device='cpu', model_save_path='./model_checkpoints_optuna/', 
                log_interval=100)  
    
    # Log to wandb
    wandb.log({'learning_rate': learning_rate,
               'batch_size': batch_size,
               'num_layers': num_layers,
               'residual_channels': residual_channels,
               'dilation_cycle_length': dilation_cycle_length,
               'val_loss': val_loss})
    
    return val_loss

In [44]:
wandb.init(project="optuna_hyperparameter_optimization")

train_loss,██▄▄▄▂▃▂▃▃▂▃▄▃▂▁▂▄▂▁▂▃▂▃▁▂▂▂▃▂▂▂▂▂▂▂▂▁▂▂
train_loss,0.07155


In [45]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

    # Complete the wandb run
wandb.run.finish()


[I 2024-03-03 19:53:04,413] A new study created in memory with name: no-name-469d8df2-0f76-4831-8b0e-70b2ac50953c


c:\Users\Admin\anaconda3\envs\difonedseg\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  This is separate from the ipykernel package so we can avoid doing imports until
100%|██████████| 156/156 [00:40<00:00,  3.83it/s, epoch=100/100, avg_loss=0.077] 
[I 2024-03-03 21:01:46,796] Trial 0 finished with value: 0.09964051761872071 and parameters: {'learning_rate': 9.945970661512229e-05, 'batch_size': 64, 'num_layers': 14, 'residual_channels': 16, 'dilation_cycle_length': 1}. Best is trial 0 with value: 0.09964051761872071.


Model saved to ./model_checkpoints_optuna/model_epoch_100.pth


100%|██████████| 156/156 [00:38<00:00,  4.05it/s, epoch=100/100, avg_loss=0.0661]
[I 2024-03-03 22:16:34,159] Trial 1 finished with value: 0.07706547251979637 and parameters: {'learning_rate': 0.0007153912999894027, 'batch_size': 64, 'num_layers': 15, 'residual_channels': 16, 'dilation_cycle_length': 2}. Best is trial 1 with value: 0.07706547251979637.


Model saved to ./model_checkpoints_optuna/model_epoch_100.pth


...

Model saved to ./model_checkpoints_optuna/model_epoch_100.pth


100%|██████████| 78/78 [00:14<00:00,  5.35it/s, epoch=100/100, avg_loss=0.105]
[I 2024-03-04 09:22:39,166] Trial 7 finished with value: 0.16249944036567132 and parameters: {'learning_rate': 2.0641558016647146e-05, 'batch_size': 128, 'num_layers': 5, 'residual_channels': 32, 'dilation_cycle_length': 1}. Best is trial 1 with value: 0.07706547251979637.


Model saved to ./model_checkpoints_optuna/model_epoch_100.pth


 88%|████████▊ | 273/312 [04:26<00:38,  1.03it/s, epoch=1/100, avg_loss=0.18] 
[W 2024-03-04 09:27:05,569] Trial 8 failed with parameters: {'learning_rate': 0.006144956989616047, 'batch_size': 32, 'num_layers': 14, 'residual_channels': 64, 'dilation_cycle_length': 4} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\Admin\anaconda3\envs\difonedseg\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Admin\AppData\local\Temp\ipykernel_25692\735566430.py", line 32, in objective
    log_interval=100)
  File "C:\Users\Admin\AppData\local\Temp\ipykernel_25692\2610101303.py", line 30, in train_model
    losses = model.log_prob(signals)
  File "c:\Users\Admin\Desktop\diffusion_ts\s\net\gaussian_diffusion.py", line 280, in log_prob
    x.reshape(B * T, 1, -1), time, *args, **kwargs
  File "c:\Users\Admin\Desktop\diffusion_ts\s\net\gaussian_diffusion.py", line 259, in p_los

KeyboardInterrupt: 

In [46]:
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

Number of finished trials:  9
Best trial:
  Value:  0.07706547251979637
  Params: 
    learning_rate: 0.0007153912999894027
    batch_size: 64
    num_layers: 15
    residual_channels: 16
    dilation_cycle_length: 2
